In [ ]:
import pandas as pd
import numpy as np
import json
import torch

from langchain_openai import ChatOpenAI,OpenAI
from langchain import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter

from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)
from huggingface_hub import login

In [ ]:
data =pd.read_csv("data/Clothing/clothingClear.csv")
data.head()

In [ ]:
data["clothingID"].sample()

In [ ]:
targetID = 1110
filteredDF = data[data["clothingID"]==targetID]
filteredDF.head(3)

In [ ]:
#filteredDF["rating"].value_counts().plot.bar()

In [ ]:
reviewList = filteredDF["cleanReview"].tolist()

In [ ]:
config_data = json.load(open("config.json"))

In [ ]:
#data ' dan target id ye göre filtrleme yapar ve reviewleri text list olarak döndürür.
def reviewListGenarator(data,targetID):
    filteredDF = data[data["clothingID"]==targetID]
    print(f"Review Count {len(filteredDF)}")
    reviewList = filteredDF["cleanReview"].tolist()
    return reviewList
    

In [ ]:
# Reviewleri birleştirir ve promt oluşturur.
def inputGenerator(reviewList):
    template = """ Analyze the user reviews provided below and create a concise paragraph summarizing the overall sentiment. Your summary should answer the following questions: Do users generally like or dislike the dress? What features are most appreciated or criticized? Do users recommend the dress? Include any additional informations observed, but keep the output as a single cohesive paragraph. Here are the user reviews:
    {revs}
    """

    prompt = PromptTemplate(
        input_variables=["revs"],
        template=template
    )
    
    text = "Review: "+" Review: ".join(reviewList)
    inputText = prompt.format(revs=text)
    return inputText

# GPT

In [ ]:

API_KEY=config_data["GPT_API_KEY"]
GPT_TOKEN_LIMIT=128000
gptModel = ChatOpenAI(model="gpt-4o-mini",openai_api_key=API_KEY)

In [ ]:
input = inputGenerator(reviewList)
input

In [ ]:
num_tokens = gptModel.get_num_tokens(input)
print (f"prompt {num_tokens} tokens")

In [ ]:
if num_tokens < GPT_TOKEN_LIMIT:
    output = gptModel.invoke(input)
    print(output.content)
else:
    print("Token boyutu cok buyuk")
    #review listten eleman dropla


# LLMA

In [ ]:
Llama_TOKEN_LIMIT=128000

# Llama-3.2 (1B): Requires 1.8 GB of GPU memory. 
# Llama-3.2 (3B): Needs 3.4 GB of GPU memory.

#model_name = "meta-llama/Llama-3.2-1B"
#model_name = "meta-llama/Llama-3.2-3B"
#LlamaModelName="meta-llama/Llama-3.2-3B-Instruct"

LlamaModelName="meta-llama/Llama-3.2-1B-Instruct"


'''
Instruction-tuned: Model, belirli görevlerde talimatları daha iyi anlamak için ince ayarlanmıştır. İnsanların verdiği talimatlara yönelik daha alakalı ve doğru sonuçlar verir.
Daha kullanıcı dostu yanıtlar üretebilmek için genellikle RLHF (Reinforcement Learning with Human Feedback) ile eğitilir.
'''

In [ ]:
HF_TOKEN = config_data["HF_TOKEN"]
login(HF_TOKEN)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
LlamaTokenizer = AutoTokenizer.from_pretrained(LlamaModelName)

LlamaTokenizer.pad_token = LlamaTokenizer.eos_token

In [ ]:
LlamaModel = AutoModelForCausalLM.from_pretrained(
    LlamaModelName,
    device_map="auto",
    quantization_config=bnb_config,
)

In [ ]:
LlamaPipe = pipeline(
    "text-generation",
    model=LlamaModel,
    tokenizer=LlamaTokenizer
)


In [ ]:
def getLlamaPipeResponse(input):
  sequences = LlamaPipe(input,return_full_text=False,
          max_new_tokens=256)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [ ]:

input = inputGenerator(reviewList)

In [ ]:
llamaTokens = LlamaTokenizer(input)
llamaTokenCount = len(llamaTokens['input_ids'])
llamaTokenCount

In [ ]:
torch.cuda.empty_cache()
getLlamaPipeResponse(input)

In [ ]:
if llamaTokenCount < Llama_TOKEN_LIMIT:
    output = getLlamaPipeResponse(input)
    print(output)
else:
    print("Token boyutu cok buyuk")
    

# Qwen

In [ ]:
# mistral 2k
#Minerva 2k
# falcon 2k
#bigscience/bloomz-1b7  CUDA out of memory. Tried to allocate 5.00 GiB. GPU 0 has a total capacity of 4.00 GiB

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

#XModelName = "Qwen/Qwen2.5-3B-Instruct"
XModelName="Qwen/Qwen2-1.5B-Instruct"
max = 32768

XModel = AutoModelForCausalLM.from_pretrained(XModelName,
                                                    device_map="auto",
                                                    quantization_config=bnb_config,
                                                    torch_dtype=torch.float16)


XTokenizer = AutoTokenizer.from_pretrained(XModelName)
XTokenizer.pad_token = XTokenizer.eos_token



In [ ]:
XPipe = pipeline(
    "text-generation",
    model=XModel,
    tokenizer=XTokenizer
)

In [ ]:
input = inputGenerator(reviewList)
tokens = XTokenizer(input)
token_count = len(tokens['input_ids'])
token_count

In [ ]:
input = inputGenerator(reviewList)

In [ ]:
torch.cuda.empty_cache()

In [ ]:

XPipe(input,return_full_text=False,max_new_tokens=256)[0]["generated_text"]